# Standard Operating Procedure for ALL-ALS Data Updates

## Step 1: Generate View Name to Class Name Mapping

This first step is to analyze the raw CSV files from the v2 data release to extract the 'Form Name' from each file. This 'Form Name' will be used to map to the class name in the data model.

In [1]:
import os
import csv
import json

def get_form_names_from_datasets(base_dir):
    datasets = ["ASSESS", "PREVENT"]
    form_name_mapping = {}

    for dataset in datasets:
        dataset_files_path = os.path.join(base_dir, dataset, 'files')
        if not os.path.isdir(dataset_files_path):
            print(f"Directory not found: {dataset_files_path}")
            continue

        for filename in os.listdir(dataset_files_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(dataset_files_path, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as csvfile:
                        reader = csv.reader(csvfile)
                        header = next(reader)
                        if "form_name" in header:
                            form_name_index = header.index("form_name")
                            try:
                                first_row = next(reader)
                                form_name = first_row[form_name_index]
                                # Clean up the filename to use as a key
                                clean_filename = os.path.splitext(filename)[0].replace('v_ALLALS_AS_', '').replace('v_ALLALS_PV_', '').replace('v_ALLALS_PR_', '').replace('-', '_')
                                form_name_mapping[clean_filename] = form_name
                            except StopIteration:
                                print(f"File is empty (after header): {filename}")
                        else:
                            print(f"'form_name' column not found in {filename}")
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

    return form_name_mapping

if __name__ == "__main__":
    # The base directory where the ASSESS and PREVENT folders are located.
    base_directory = "/home/ramayyala/Documents/data-model/data/ALL_ALS/v3-DEC"
    mappings = get_form_names_from_datasets(base_directory)

    # Save the mappings to a file in the root of the project
    output_file_path = "/home/ramayyala/Documents/data-model/form_name_mappings.json"
    with open(output_file_path, 'w') as f:
        json.dump(mappings, f, indent=4)

    print(f"Mappings saved to {output_file_path}")

    # Print the mappings to the console as well
    for key, value in mappings.items():
        print(f"{key}: {value}")

'form_name' column not found in v_ALLALS_PR_PREVOGCVH.csv
Mappings saved to /home/ramayyala/Documents/data-model/form_name_mappings.json
ASSEAELOG: Adverse Event (AE) Log
ASSEALSFRSR: ALSFRS-R
ASSEALSFRSRSAQ: ALSFRS-R and Speech Anchor Questions
ASSEALSHIST: ALS History
ASSEBLDCOL: Blood Collection (ASSESS)
ASSECONMD: Concomitant Medication Log
ASSECSFCO: Cerebral Spinal Fluid (CSF) Collection
ASSEDEMOG: Demographics
ASSEDSA: Digital Speech Assessment
ASSEDSAUI: Digital Speech Assessment User Information
ASSEECASCGI: ECAS (Edinburgh Cognitive and Behavioral ALS Screen) - Caregiver Interview
ASSEECAS: ECAS (Edinburgh Cognitive and Behavioral ALS Screen)
ASSEELCONLMP: Lumbar Puncture Eligibility Confirmation
ASSEELCONREG: Enrollment Confirmation
ASSEELIGCRIREG2: Eligibility Criteria (ASSESS Control)
ASSEELIGCRISYM2: Eligibility Criteria (ASSESS Symptomatic)
ASSEELIGCRITLMP: Lumbar Puncture Eligibility Criteria
ASSEELIGCRITREG: Eligibility Criteria (ASSESS Control)
ASSEELIGCRITSYM: Eligib

## Step 2: Configuration Setup

Configure parameters for the annotation and upload workflow including Synapse authentication, dataset IDs, version labels, and workflow control settings.

**IMPORTANT:** Set `DRY_RUN = True` for initial testing to preview changes without uploading.


In [ ]:
# ==================== CONFIGURATION ====================
# USER: Update these values before running the workflow

# Synapse Authentication
SYNAPSE_AUTH_TOKEN = ""

## OLD RELEASE DATASETS (for pulling existing annotations)
# Dataset Synapse IDs (existing v2-OCT datasets to pull annotations from)
ASSESS_DATASET_SYN_ID = "syn69694463"  
PREVENT_DATASET_SYN_ID = "syn69694674"  

# Staging Folders (where contributors upload new file versions)
# Set these to empty strings "" to fall back to local file workflow
ASSESS_STAGING_FOLDER_SYN_ID = "syn72119685"  # TODO: Provide ASSESS staging folder syn ID
PREVENT_STAGING_FOLDER_SYN_ID = "syn72119684"  # TODO: Provide PREVENT staging folder syn ID

# Release Folders (where final release files are stored)
# Required if staging folders are provided
ASSESS_RELEASE_FOLDER_SYN_ID = "syn68885185"  # TODO: Provide ASSESS release folder syn ID
PREVENT_RELEASE_FOLDER_SYN_ID = "syn68885187"  # TODO: Provide PREVENT release folder syn ID

# Project ID for uploads
SYNAPSE_PROJECT_ID = "syn68702804"

# Version information
VERSION_LABEL = "v3-DEC"
VERSION_COMMENT = "Dec Release"

# ==================== AUTO-CONFIGURED PATHS ====================
# These paths are automatically configured based on the structure

BASE_DIR = "/home/ramayyala/Documents/data-model"
DATA_DIR = os.path.join(BASE_DIR, "data/ALL_ALS/v3-DEC")
ASSESS_FILES_DIR = os.path.join(DATA_DIR, "ASSESS/files")
PREVENT_FILES_DIR = os.path.join(DATA_DIR, "PREVENT/files")

# Model schemas directory
SCHEMA_BASE_PATH = os.path.join(BASE_DIR, "model_schemas")

# Form name mappings (generated by Cell 2)
FORM_MAPPINGS_FILE = os.path.join(BASE_DIR, "form_name_mappings.json")

# Output annotation files
ANNOTATIONS_DIR = os.path.join(BASE_DIR, "annotations/all_als")
os.makedirs(ANNOTATIONS_DIR, exist_ok=True)

ASSESS_ANNOTATIONS_FILE = os.path.join(ANNOTATIONS_DIR, "assess_file_annotations.json")
PREVENT_ANNOTATIONS_FILE = os.path.join(ANNOTATIONS_DIR, "prevent_file_annotations.json")
ASSESS_DATASET_ANNOTATIONS_FILE = os.path.join(ANNOTATIONS_DIR, "assess_dataset_annotations.json")
PREVENT_DATASET_ANNOTATIONS_FILE = os.path.join(ANNOTATIONS_DIR, "prevent_dataset_annotations.json")

# Staging directories for renamed files
STAGING_DIR = os.path.join(BASE_DIR, "staging/all_als")
ASSESS_STAGING_DIR = os.path.join(STAGING_DIR, "assess")
PREVENT_STAGING_DIR = os.path.join(STAGING_DIR, "prevent")
os.makedirs(ASSESS_STAGING_DIR, exist_ok=True)
os.makedirs(PREVENT_STAGING_DIR, exist_ok=True)

# Download directory for Synapse files
DOWNLOAD_DIR = os.path.join(BASE_DIR, "downloads/all_als")
ASSESS_DOWNLOAD_DIR = os.path.join(DOWNLOAD_DIR, "assess")
PREVENT_DOWNLOAD_DIR = os.path.join(DOWNLOAD_DIR, "prevent")
os.makedirs(ASSESS_DOWNLOAD_DIR, exist_ok=True)
os.makedirs(PREVENT_DOWNLOAD_DIR, exist_ok=True)

# ==================== WORKFLOW CONTROL ====================
DRY_RUN = False  # Set to False to actually upload files
VERBOSE = True  # Set to False for less output

# Determine workflow mode
USE_SYNAPSE_STAGING = bool(ASSESS_STAGING_FOLDER_SYN_ID and PREVENT_STAGING_FOLDER_SYN_ID)

# ==================== VALIDATION ====================
print("Configuration Validation:")
print("-" * 50)

if not SYNAPSE_AUTH_TOKEN:
    print("⚠️  WARNING: SYNAPSE_AUTH_TOKEN not set")
if not ASSESS_DATASET_SYN_ID:
    print("⚠️  WARNING: ASSESS_DATASET_SYN_ID not set")
if not PREVENT_DATASET_SYN_ID:
    print("⚠️  WARNING: PREVENT_DATASET_SYN_ID not set")
if not SYNAPSE_PROJECT_ID:
    print("⚠️  WARNING: SYNAPSE_PROJECT_ID not set")

# Validate staging/release folder configuration
if USE_SYNAPSE_STAGING:
    print(f"✓ Synapse-based workflow enabled")
    if not ASSESS_RELEASE_FOLDER_SYN_ID or not PREVENT_RELEASE_FOLDER_SYN_ID:
        print("❌ ERROR: Release folder IDs required when staging folders are provided")
        print("   Please set ASSESS_RELEASE_FOLDER_SYN_ID and PREVENT_RELEASE_FOLDER_SYN_ID")
        raise ValueError("Release folder IDs required with staging folders")
    print(f"  - ASSESS staging: {ASSESS_STAGING_FOLDER_SYN_ID}")
    print(f"  - PREVENT staging: {PREVENT_STAGING_FOLDER_SYN_ID}")
    print(f"  - ASSESS release: {ASSESS_RELEASE_FOLDER_SYN_ID}")
    print(f"  - PREVENT release: {PREVENT_RELEASE_FOLDER_SYN_ID}")
else:
    print(f"✓ Local file workflow (staging folders not configured)")

print(f"✓ Base directory: {BASE_DIR}")
print(f"✓ Data directory: {DATA_DIR}")
print(f"✓ Annotations directory: {ANNOTATIONS_DIR}")
print(f"✓ Staging directory: {STAGING_DIR}")
print(f"✓ Download directory: {DOWNLOAD_DIR}")
print(f"✓ Version: {VERSION_LABEL}")
print(f"✓ DRY_RUN mode: {DRY_RUN}")
print("-" * 50)


Configuration Validation:
--------------------------------------------------
✓ Synapse-based workflow enabled
  - ASSESS staging: syn72119685
  - PREVENT staging: syn72119684
  - ASSESS release: syn68885185
  - PREVENT release: syn68885187
✓ Base directory: /home/ramayyala/Documents/data-model
✓ Data directory: /home/ramayyala/Documents/data-model/data/ALL_ALS/v3-DEC
✓ Annotations directory: /home/ramayyala/Documents/data-model/annotations/all_als
✓ Staging directory: /home/ramayyala/Documents/data-model/staging/all_als
✓ Download directory: /home/ramayyala/Documents/data-model/downloads/all_als
✓ Version: v3-DEC
✓ DRY_RUN mode: False
--------------------------------------------------


## Step 3: Library Imports

Import required Python libraries for the workflow.


In [12]:
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple
import shutil
from pprint import pprint

# Synapse client
import synapseclient
from synapseclient.models import (
    File, Folder, Project, Table, EntityView, Dataset,
    DatasetCollection, MaterializedView, SubmissionView
)

# Schema handling
import yaml
from jsonschema import validate, ValidationError, Draft7Validator
import re
print("✓ All libraries imported successfully")


✓ All libraries imported successfully


## Step 4: Core Utility Functions

Define utility functions for Synapse connection, schema loading, annotation cleaning, and validation.


In [13]:
# ==================== SYNAPSE CONNECTION ====================

def connect_to_synapse():
    """Connect to Synapse using the configured auth token."""
    if not SYNAPSE_AUTH_TOKEN:
        raise ValueError("SYNAPSE_AUTH_TOKEN is not set. Please update configuration.")

    try:
        syn = synapseclient.Synapse()
        syn.login(authToken=SYNAPSE_AUTH_TOKEN)
        print("✓ Successfully connected to Synapse")
        return syn
    except Exception as e:
        print(f"✗ Failed to connect to Synapse: {e}")
        raise


# ==================== SCHEMA LOADING ====================

def get_all_schemas(schema_base_path=None):
    """Load all YAML schema files recursively."""
    if schema_base_path is None:
        schema_base_path = SCHEMA_BASE_PATH

    schemas = {}
    schema_path = Path(schema_base_path)

    for yaml_file in schema_path.rglob("*.yaml"):
        try:
            with open(yaml_file, 'r') as f:
                schema_data = yaml.safe_load(f)
                if schema_data and isinstance(schema_data, dict):
                    # Look for 'classes' key in YAML structure
                    if 'classes' in schema_data:
                        for class_name, class_def in schema_data['classes'].items():
                            if isinstance(class_def, dict):
                                schemas[class_name] = class_def
                    # Also support old flat structure for backwards compatibility
                    else:
                        for class_name, class_def in schema_data.items():
                            if isinstance(class_def, dict):
                                schemas[class_name] = class_def
        except Exception as e:
            if VERBOSE:
                print(f"Warning: Could not load {yaml_file}: {e}")

    print(f"✓ Loaded {len(schemas)} schema definitions")
    return schemas


def get_full_schema(class_name, all_schemas, visited=None):
    """Get full schema for a class including inherited attributes."""
    if visited is None:
        visited = set()

    if class_name in visited:
        return {}
    visited.add(class_name)

    if class_name not in all_schemas:
        return {}

    schema = all_schemas[class_name]
    full_attributes = {}

    # Process inheritance (is_a)
    if 'is_a' in schema:
        parent_class = schema['is_a']
        parent_attrs = get_full_schema(parent_class, all_schemas, visited)
        full_attributes.update(parent_attrs)

    # Process mixins
    if 'mixins' in schema and isinstance(schema['mixins'], list):
        for mixin in schema['mixins']:
            mixin_attrs = get_full_schema(mixin, all_schemas, visited)
            full_attributes.update(mixin_attrs)

    # Add own attributes
    if 'attributes' in schema:
        full_attributes.update(schema['attributes'])

    return full_attributes


# ==================== ANNOTATION CLEANING ====================

def clean_annotations_for_synapse(annotation):
    """Remove metadata fields and empty values before applying to Synapse."""
    cleaned = {}

    for key, value in annotation.items():
        # Skip metadata fields (starting with _)
        if key.startswith('_'):
            continue
        
        # Convert File objects to syn IDs (do this FIRST before any comparisons)
        if hasattr(value, '__class__') and value.__class__.__name__ == 'File':
            value = value.id if hasattr(value, 'id') else str(value)
        
        # Handle lists that might contain File objects
        elif isinstance(value, list):
            cleaned_list = []
            for item in value:
                if hasattr(item, '__class__') and item.__class__.__name__ == 'File':
                    cleaned_list.append(item.id if hasattr(item, 'id') else str(item))
                else:
                    cleaned_list.append(item)
            value = cleaned_list
        
        # NOW check for truly empty values (after conversion)
        if value is None:
            continue
        if isinstance(value, str) and value == "":
            continue
        if isinstance(value, list) and (len(value) == 0 or (len(value) == 1 and value[0] == "")):
            continue

        cleaned[key] = value

    return cleaned


# ==================== VALIDATION ====================

def validate_annotation_against_schema(annotation, file_type, all_schemas):
    """Validate a single annotation against its schema."""
    errors = []
    warnings = []

    # Get full schema with inheritance
    schema_attributes = get_full_schema(file_type, all_schemas)

    if not schema_attributes:
        errors.append(f"Schema not found for file type: {file_type}")
        return False, errors, warnings

    # Check required fields
    for attr_name, attr_def in schema_attributes.items():
        is_required = attr_def.get('required', False)

        if is_required:
            if attr_name not in annotation:
                errors.append(f"Required field missing: {attr_name}")
            elif annotation[attr_name] in ["", [""], [], None]:
                errors.append(f"Required field empty: {attr_name}")

    # Check multivalued fields
    for attr_name, attr_value in annotation.items():
        if attr_name.startswith('_'):
            continue

        if attr_name in schema_attributes:
            attr_def = schema_attributes[attr_name]
            is_multivalued = attr_def.get('multivalued', False)

            if is_multivalued and not isinstance(attr_value, list):
                errors.append(f"Field {attr_name} should be a list")
            elif not is_multivalued and isinstance(attr_value, list):
                warnings.append(f"Field {attr_name} is a list but should be a single value")

    # Check if at least some fields are filled
    filled_fields = sum(1 for k, v in annotation.items()
                       if not k.startswith('_') and v not in ["", [""], [], None])

    if filled_fields == 0:
        warnings.append("No annotation fields are filled")

    is_valid = len(errors) == 0
    return is_valid, errors, warnings


# ==================== NEW SYNAPSE-BASED WORKFLOW FUNCTIONS ====================

def enumerate_folder_files(syn, folder_syn_id):
    """
    Query all files in a Synapse folder and extract their annotations.
    
    Args:
        syn: Synapse client
        folder_syn_id: Synapse ID of the folder
    
    Returns: {syn_id: {filename: {annotations}}}
    """
    print(f"Retrieving files from folder {folder_syn_id}...")
    
    try:
        # Query for all files in the folder
        query_results = list(syn.getChildren(folder_syn_id, includeTypes=["file"]))
        
        annotations_dict = {}
        file_count = 0
        
        for item in query_results:
            syn_id = item['id']
            filename = item['name']
            
            # Get annotations for this file
            try:
                entity = syn.get(syn_id, downloadFile=False)
                annotations = dict(entity.annotations) if hasattr(entity, 'annotations') else {}
                
                if annotations or True:  # Include all files even without annotations
                    annotations_dict[syn_id] = {filename: annotations}
                    file_count += 1
                    
            except Exception as e:
                if VERBOSE:
                    print(f"  ⚠️  Could not get annotations for {filename}: {e}")
        
        print(f"✓ Retrieved {file_count} files from folder")
        return annotations_dict
        
    except Exception as e:
        print(f"✗ Error retrieving folder files: {e}")
        return {}


def download_synapse_file(syn, file_syn_id, download_dir):
    """
    Download a file from Synapse to local directory.
    
    Args:
        syn: Synapse client
        file_syn_id: Synapse ID of the file
        download_dir: Local directory to download to
    
    Returns: Local file path or None if error
    """
    try:
        entity = syn.get(file_syn_id, downloadLocation=download_dir)
        local_path = entity.path
        
        if VERBOSE:
            print(f"  ✓ Downloaded {entity.name} to {local_path}")
        
        return local_path
        
    except Exception as e:
        print(f"  ✗ Error downloading {file_syn_id}: {e}")
        return None


def merge_file_annotations(old_annot, new_annot, template):
    """
    Merge annotations with priority: old (release) > new (staging) > template.
    
    Args:
        old_annot: Annotations from release version (highest priority)
        new_annot: Annotations from staging version
        template: Template with all schema fields
    
    Returns: Merged annotation dict
    """
    merged = {}
    
    # Start with template to ensure all fields are present
    merged.update(template)
    
    # Add new staging annotations (overwrite template)
    for key, value in new_annot.items():
        if value not in ["", [""], [], None]:
            merged[key] = value
    
    # Add old release annotations (overwrite template and new - highest priority)
    for key, value in old_annot.items():
        if value not in ["", [""], [], None]:
            merged[key] = value
    
    return merged


def move_file_to_release(syn, file_syn_id, release_folder_syn_id, dataset_syn_id, annotations, form_name, dry_run=True):
    """
    Move file from staging folder to release folder, add to dataset, and set annotations.
    
    Args:
        syn: Synapse client
        file_syn_id: Synapse ID of file to move
        release_folder_syn_id: Synapse ID of release folder (destination)
        dataset_syn_id: Synapse ID of dataset to add file to
        annotations: Annotations to apply to the file
        form_name: Form name to rename the file to (e.g., "Demographics")
        dry_run: If True, only preview the operation
    
    Returns: (success, error_message)
    """
    try:
        # Get file entity info
        file_entity = syn.get(file_syn_id, downloadFile=False)
        filename = file_entity.name
        
        if dry_run:
            print(f"  [DRY_RUN] Would move {filename}:")
            print(f"    - From: current folder → {release_folder_syn_id}")
            print(f"    - Rename to: {form_name}.csv")
            print(f"    - Add to dataset: {dataset_syn_id}")
            print(f"    - Set annotations: {len(clean_annotations_for_synapse(annotations))} fields")
            return (True, None)
        
        # Step 1: Move file to release folder
        try:
            syn.move(file_syn_id, release_folder_syn_id)
            if VERBOSE:
                print(f"  ✓ Moved {filename} to release folder")
        except Exception as e:
            raise Exception(f"Failed to move file: {e}")
        
        # Step 1.5: Rename file using form name
        try:
            file_entity = syn.get(file_syn_id, downloadFile=False)
            original_filename = file_entity.name
            # Sanitize the form name to comply with Synapse naming rules
            sanitized_form_name = sanitize_synapse_filename(form_name)
            new_filename = f"{sanitized_form_name}.csv"
            
            if original_filename != new_filename:
                file_entity.name = new_filename
                syn.store(file_entity, forceVersion=False)
                if VERBOSE:
                    print(f"  ✓ Renamed {original_filename} → {new_filename}")
        except Exception as e:
            print(f"  ⚠️  Warning: Could not rename file: {e}")
            # Don't fail - file is already moved
        
        # Step 2: Add file to dataset
        try:
            dataset = Dataset(dataset_syn_id).get()
            # Add file to dataset items
            from synapseclient.models import File as FileModel
            file_to_add = FileModel(id=file_syn_id)
            dataset.add_item(file_to_add)
            dataset.store()
            if VERBOSE:
                print(f"  ✓ Added {filename} to dataset")
        except Exception as e:
            print(f"  ⚠️  Warning: Could not add to dataset: {e}")
            # Don't fail - file is already moved
        
        # Step 3: Set annotations on the moved file
        try:
            # Re-fetch the file entity after move to avoid 412 conflict
            file_entity = syn.get(file_syn_id, downloadFile=False)
            cleaned_annotations = clean_annotations_for_synapse(annotations)
            file_entity.annotations = cleaned_annotations
            syn.store(file_entity, forceVersion=False)
            if VERBOSE:
                print(f"  ✓ Set annotations on {filename}")
        except Exception as e:
            print(f"  ⚠️  Warning: Could not set annotations: {e}")
            # Don't fail - file is moved and in dataset
        
        return (True, None)
        
    except Exception as e:
        # Handle case where filename isn't set yet
        file_desc = filename if 'filename' in locals() else file_syn_id
        error_msg = f"Failed to move {file_desc}: {e}"
        print(f"  ✗ {error_msg}")
        return (False, error_msg)


print("✓ Core utility functions defined (including Synapse workflow functions)")


✓ Core utility functions defined (including Synapse workflow functions)


## Step 5: Retrieve Existing Annotations from Synapse

Pull existing annotations from ASSESS and PREVENT dataset entities. This retrieves annotations from v2-OCT datasets to understand what files already exist and their current metadata.

**Output Structure:** `{syn_id: {file_name: {annotations}}}`


In [14]:
def get_existing_synapse_annotations(syn, entity_id):
    """Get annotations from a single Synapse entity."""
    try:
        entity = syn.get(entity_id, downloadFile=False)
        return [entity.name,dict(entity.annotations)]
    except Exception as e:
        if VERBOSE:
            print(f"Warning: Could not get annotations for {entity_id}: {e}")
        return {}

def enumerate_dataset_files(syn, dataset_syn_id):
    """
    Query all files in a dataset and extract their annotations.
    Returns: {syn_id: {filename: {annotations}}}
    """
    print(f"Retrieving files from dataset {dataset_syn_id}...")

    try:
        # Get the dataset entity
        dataset = Dataset(dataset_syn_id).get()
        results=dataset.items
        annotations_dict = {}
        file_count = 0

        for row in results:
             syn_id=row.id
            # Get annotations for this file
             results = get_existing_synapse_annotations(syn, syn_id)
             filename=results[0]
             annotations=results[1]
             if annotations:
                annotations_dict[syn_id] = {filename: annotations}
                file_count += 1

        print(f"✓ Retrieved annotations for {file_count} files")
        return annotations_dict

    except Exception as e:
        print(f"✗ Error retrieving dataset files: {e}")
        print("Note: Make sure you have provided a valid dataset syn ID")
        return {}


def get_dataset_annotations(syn, dataset_syn_id):
    """Get annotations from the dataset entity itself."""
    try:
        dataset = syn.get(dataset_syn_id, downloadFile=False)
        return dict(dataset.annotations) if hasattr(dataset, 'annotations') and dataset.annotations else {}
    except Exception as e:
        print(f"⚠️  Could not get dataset annotations: {e}")
        return {}


# Connect to Synapse
print("=" * 60)
print("RETRIEVING EXISTING ANNOTATIONS FROM SYNAPSE")
print("=" * 60)

if not ASSESS_DATASET_SYN_ID or not PREVENT_DATASET_SYN_ID:
    print("⚠️  Skipping: Dataset syn IDs not configured")
    print("Please update ASSESS_DATASET_SYN_ID and PREVENT_DATASET_SYN_ID in the configuration cell")
    assess_existing_annotations = {}
    prevent_existing_annotations = {}
    assess_dataset_existing_annotations = {}
    prevent_dataset_existing_annotations = {}
    assess_staging_files = {}
    prevent_staging_files = {}
else:
    syn = connect_to_synapse()

    print("\n--- RELEASE DATASETS (Old Versions) ---")
    print("\nASSESS Dataset:")
    # Get file annotations from release dataset
    assess_existing_annotations = enumerate_dataset_files(syn, ASSESS_DATASET_SYN_ID)
    # Get dataset entity annotations
    assess_dataset_existing_annotations = get_dataset_annotations(syn, ASSESS_DATASET_SYN_ID)

    print("\nPREVENT Dataset:")
    # Get file annotations from release dataset
    prevent_existing_annotations = enumerate_dataset_files(syn, PREVENT_DATASET_SYN_ID)
    # Get dataset entity annotations
    prevent_dataset_existing_annotations = get_dataset_annotations(syn, PREVENT_DATASET_SYN_ID)

    # Retrieve from staging folders if configured
    if USE_SYNAPSE_STAGING:
        print("\n--- STAGING FOLDERS (New Versions) ---")
        
        print("\nASSESS Staging Folder:")
        assess_staging_files = enumerate_folder_files(syn, ASSESS_STAGING_FOLDER_SYN_ID)
        
        print("\nPREVENT Staging Folder:")
        prevent_staging_files = enumerate_folder_files(syn, PREVENT_STAGING_FOLDER_SYN_ID)
    else:
        print("\n⚠️  Staging folders not configured - using local file workflow")
        assess_staging_files = {}
        prevent_staging_files = {}

    print("\n" + "=" * 60)
    print("RETRIEVAL SUMMARY")
    print("=" * 60)
    print(f"ASSESS:")
    print(f"  - Release files: {len(assess_existing_annotations)}")
    print(f"  - Staging files: {len(assess_staging_files)}")
    print(f"  - Dataset annotations: {len(assess_dataset_existing_annotations)} fields")
    print(f"\nPREVENT:")
    print(f"  - Release files: {len(prevent_existing_annotations)}")
    print(f"  - Staging files: {len(prevent_staging_files)}")
    print(f"  - Dataset annotations: {len(prevent_dataset_existing_annotations)} fields")
    print(f"\nWorkflow mode: {'Synapse-based' if USE_SYNAPSE_STAGING else 'Local files'}")
    print("=" * 60)


RETRIEVING EXISTING ANNOTATIONS FROM SYNAPSE

UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.10.0) is available. Your version (4.8.0) can be upgraded by typing:
   pip install --upgrade synapseclient

Python Synapse Client version 4.10.0 release notes

https://python-docs.synapse.org/news/


Welcome, ram.ayyala!

✓ Successfully connected to Synapse

--- RELEASE DATASETS (Old Versions) ---

ASSESS Dataset:
Retrieving files from dataset syn69694463...
✓ Retrieved annotations for 32 files

PREVENT Dataset:
Retrieving files from dataset syn69694674...
✓ Retrieved annotations for 49 files

--- STAGING FOLDERS (New Versions) ---

ASSESS Staging Folder:
Retrieving files from folder syn72119685...
✓ Retrieved 32 files from folder

PREVENT Staging Folder:
Retrieving files from folder syn72119684...
✓ Retrieved 50 files from folder

RETRIEVAL SUMMARY
ASSESS:
  - Release files: 32
  - Staging files: 32
  - Dataset annotations: 12 fields

PREVENT:
  - Release files: 49
  - Stagi

## Step 6: Compare Local Files with Existing Annotations

Compare local v3-DEC files with existing Synapse annotations to identify:
1. **Matched files**: Exist in Synapse (will be updated with new versions)
2. **Unmatched files**: New files needing annotation templates

Matching uses form_name_mappings.json to normalize filenames.


In [17]:
def load_form_name_mappings():
    """Load the form_name_mappings.json file."""
    if not os.path.exists(FORM_MAPPINGS_FILE):
        print(f"⚠️  Warning: {FORM_MAPPINGS_FILE} not found")
        print("Please run Cell 2 to generate the mapping file")
        return {}

    with open(FORM_MAPPINGS_FILE, 'r') as f:
        return json.load(f)


def get_local_files(directory):
    """Get list of CSV files in directory with full paths."""
    if not os.path.isdir(directory):
        print(f"⚠️  Warning: Directory not found: {directory}")
        return []

    files = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            files.append((filename, file_path))

    return files


def extract_clean_filename(filename):
    """Remove v_ALLALS_XX_ prefix from filename."""
    clean = filename.replace('v_ALLALS_AS_', '').replace('v_ALLALS_PV_', '').replace('v_ALLALS_PR_', '')
    clean = os.path.splitext(clean)[0]  # Remove .csv extension
    clean = clean.replace('-', '_')
    return clean




def sanitize_synapse_filename(filename):
    """
    Sanitize filename to comply with Synapse naming rules.
    
    Synapse allows only: letters, numbers, spaces, underscores, hyphens, 
    periods, plus signs, apostrophes, and parenthesis.
    
    Args:
        filename: Original filename
    
    Returns:
        Sanitized filename safe for Synapse
    """
    # Replace common problematic characters with safe alternatives
    replacements = {
        ':': '-',  # Colon to hyphen
        '/': '-',  # Slash to hyphen
        '\\': '-',  # Backslash to hyphen
        '|': '-',  # Pipe to hyphen
        '*': '',   # Asterisk removed
        '?': '',   # Question mark removed
        '"': "'",  # Double quote to single quote
        '<': '(',  # Less than to open paren
        '>': ')',  # Greater than to close paren
    }
    
    sanitized = filename
    for old_char, new_char in replacements.items():
        sanitized = sanitized.replace(old_char, new_char)
    
    # Remove any remaining characters that aren't allowed
    # Allowed: letters, numbers, spaces, _-.()+' and ()
    sanitized = re.sub(r"[^a-zA-Z0-9 _\\-\\.\\+\'\\(\\)]", "", sanitized)
    
    # Clean up multiple spaces or hyphens in a row
    sanitized = re.sub(r' +', ' ', sanitized)
    sanitized = re.sub(r'-+', '-', sanitized)
    
    # Remove leading/trailing spaces and hyphens
    sanitized = sanitized.strip(' -')
    
    return sanitized

def match_three_way(staging_files, release_files, local_files, form_mappings, dataset_prefix=None):
    """
    Three-way matching: staging ↔ release ↔ local (fallback).
    
    Args:
        staging_files: {syn_id: {filename: {annotations}}} from staging folder
        release_files: {syn_id: {filename: {annotations}}} from release dataset
        local_files: [(filename, path)] from local directory
        form_mappings: {clean_filename: form_name}
        dataset_prefix: Optional prefix to filter form_mappings (e.g., "ASSE" or "PREV")
    
    Returns:
        matched: List[(staging_syn_id, release_syn_id, local_path, form_name)]
        new_only: List[(staging_syn_id, staging_filename, form_name)]
        no_staging: List[(release_syn_id, release_filename)]
    """
    matched = []
    new_only = []
    no_staging = []
    
    # Filter form_mappings by dataset prefix if provided
    if dataset_prefix:
        filtered_mappings = {k: v for k, v in form_mappings.items() if k.startswith(dataset_prefix)}
        if not filtered_mappings:
            print(f"⚠️  Warning: No form mappings found for prefix '{dataset_prefix}'")
            filtered_mappings = form_mappings
    else:
        filtered_mappings = form_mappings
    
    # Create lookup: form_name → (release_syn_id, release_filename)
    form_name_to_release = {}
    for release_syn_id, file_data in release_files.items():
        for release_filename in file_data.keys():
            # Remove .csv extension and sanitize for comparison
            release_base = release_filename.replace('.csv', '').replace('.CSV', '')
            release_sanitized = sanitize_synapse_filename(release_base)
            # Normalize for comparison (treat hyphens and underscores the same)
            release_normalized = release_sanitized.replace('_', '-').lower().strip()
            
            # Try to match by form name
            for clean_key, form_name in filtered_mappings.items():
                form_sanitized = sanitize_synapse_filename(form_name)
                form_normalized = form_sanitized.replace('_', '-').lower().strip()
                
                # Try exact match after normalization
                if (release_normalized == form_normalized or 
                    release_sanitized == form_sanitized or 
                    release_base == form_name or 
                    release_filename == f"{form_name}.csv" or
                    clean_key in release_filename):
                    form_name_to_release[form_name] = (release_syn_id, release_filename)
                    break
    
    # Create lookup: form_name → (local_filename, local_path)
    form_name_to_local = {}
    for local_filename, local_path in local_files:
        clean_name = extract_clean_filename(local_filename)
        form_name = filtered_mappings.get(clean_name, clean_name)
        form_name_to_local[form_name] = (local_filename, local_path)
    
    # If we have staging files, match them to release
    if staging_files:
        # Track which release files we've matched
        matched_release_syn_ids = set()
        
        for staging_syn_id, file_data in staging_files.items():
            for staging_filename in file_data.keys():
                # Extract form name from staging filename
                clean_name = extract_clean_filename(staging_filename)
                form_name = filtered_mappings.get(clean_name, clean_name)
                
                # Try to find match in release
                if form_name in form_name_to_release:
                    release_syn_id, release_filename = form_name_to_release[form_name]
                    matched_release_syn_ids.add(release_syn_id)
                    
                    # Get local path (may be None if not in local)
                    local_path = None
                    if form_name in form_name_to_local:
                        _, local_path = form_name_to_local[form_name]
                    
                    matched.append((staging_syn_id, release_syn_id, local_path, form_name))
                else:
                    # File in staging but not in release - needs to be moved
                    new_only.append((staging_syn_id, staging_filename, form_name))
        
        # Find release files with no staging match
        for release_syn_id, file_data in release_files.items():
            if release_syn_id not in matched_release_syn_ids:
                for release_filename in file_data.keys():
                    no_staging.append((release_syn_id, release_filename))
    
    else:
        # Fallback to local file matching (original workflow)
        for local_filename, local_path in local_files:
            clean_name = extract_clean_filename(local_filename)
            form_name = filtered_mappings.get(clean_name, clean_name)
            
            # Try to find match in release
            if form_name in form_name_to_release:
                release_syn_id, release_filename = form_name_to_release[form_name]
                # Use None for staging_syn_id to indicate local-only workflow
                matched.append((None, release_syn_id, local_path, form_name))
            else:
                # New file (no staging_syn_id, no release_syn_id)
                new_only.append((None, local_filename, form_name))
    
    return matched, new_only, no_staging


# Load form name mappings
print("=" * 60)
print("MATCHING FILES (THREE-WAY)")
print("=" * 60)

form_mappings = load_form_name_mappings()
print(f"✓ Loaded {len(form_mappings)} form name mappings\n")

# Get local files
print("Scanning local files...")
assess_local_files = get_local_files(ASSESS_FILES_DIR)
prevent_local_files = get_local_files(PREVENT_FILES_DIR)

print(f"✓ Found {len(assess_local_files)} ASSESS local files")
print(f"✓ Found {len(prevent_local_files)} PREVENT local files\n")

# Three-way matching
print("Performing three-way matching...")
print(f"Mode: {'Synapse staging → release → local' if USE_SYNAPSE_STAGING else 'Local → release'}\n")

assess_matched, assess_new_only, assess_no_staging = match_three_way(
    assess_staging_files, assess_existing_annotations, assess_local_files, form_mappings, "ASSE"
)

prevent_matched, prevent_new_only, prevent_no_staging = match_three_way(
    prevent_staging_files, prevent_existing_annotations, prevent_local_files, form_mappings, "PREV"
)

print("\n" + "=" * 60)
print("MATCHING RESULTS")
print("=" * 60)
print(f"ASSESS:")
print(f"  - Matched (will update): {len(assess_matched)} files")
print(f"  - New files (need move/create): {len(assess_new_only)} files")
if USE_SYNAPSE_STAGING:
    print(f"  - In release but not staging: {len(assess_no_staging)} files (will skip)")

print(f"\nPREVENT:")
print(f"  - Matched (will update): {len(prevent_matched)} files")
print(f"  - New files (need move/create): {len(prevent_new_only)} files")
if USE_SYNAPSE_STAGING:
    print(f"  - In release but not staging: {len(prevent_no_staging)} files (will skip)")

if assess_new_only:
    print(f"\n{'📁' if USE_SYNAPSE_STAGING else '⚠️'}  New ASSESS files:")
    for staging_syn_id, filename, form_name in assess_new_only:
        if staging_syn_id:
            print(f"  - {filename} ({staging_syn_id}) → will MOVE to release")
        else:
            print(f"  - {filename} → will CREATE (needs syn ID)")

if prevent_new_only:
    print(f"\n{'📁' if USE_SYNAPSE_STAGING else '⚠️'}  New PREVENT files:")
    for staging_syn_id, filename, form_name in prevent_new_only:
        if staging_syn_id:
            print(f"  - {filename} ({staging_syn_id}) → will MOVE to release")
        else:
            print(f"  - {filename} → will CREATE (needs syn ID)")

print("=" * 60)


MATCHING FILES (THREE-WAY)
✓ Loaded 82 form name mappings

Scanning local files...
✓ Found 32 ASSESS local files
✓ Found 50 PREVENT local files

Performing three-way matching...
Mode: Synapse staging → release → local


MATCHING RESULTS
ASSESS:
  - Matched (will update): 30 files
  - New files (need move/create): 2 files
  - In release but not staging: 4 files (will skip)

PREVENT:
  - Matched (will update): 42 files
  - New files (need move/create): 8 files
  - In release but not staging: 9 files (will skip)

📁  New ASSESS files:
  - v_ALLALS_AS_ASSEICFCSF.csv (syn71824528) → will MOVE to release
  - v_ALLALS_AS_NPROREGASSESS.csv (syn71824537) → will MOVE to release

📁  New PREVENT files:
  - v_ALLALS_PR_NPROREG.csv (syn71824458) → will MOVE to release
  - v_ALLALS_PR_PREVFUTELEV.csv (syn71824478) → will MOVE to release
  - v_ALLALS_PR_PREVHHD.csv (syn71824485) → will MOVE to release
  - v_ALLALS_PR_PREVICFGEN.csv (syn71824484) → will MOVE to release
  - v_ALLALS_PR_PREVOGCVH.csv (syn

## Step 7: Generate Annotation Templates

Generate annotation JSON files for manual editing:
- **File annotations**: `{syn_id: {file_name: {annotations}}}`
- **Dataset annotations**: Flat dictionary

For matched files: Uses existing syn_id and merges with schema templates
For new files: Uses placeholder "NEW_{clean_filename}" as syn_id

**User will manually edit these files in the next step.**


In [18]:
def create_file_annotation_template(file_type='ClinicalFile'):
    """Generate empty annotation template from schema."""
    all_schemas = get_all_schemas()
    schema_attributes = get_full_schema(file_type, all_schemas)

    template = {}
    for attr_name, attr_def in schema_attributes.items():
        if attr_def.get('multivalued', False):
            template[attr_name] = ['']
        else:
            template[attr_name] = ''

    template['_file_type'] = file_type
    template['_schema_source'] = 'data-model'
    template['_created_timestamp'] = datetime.now().isoformat()

    return template


def create_dataset_annotation_template(dataset_type='ClinicalDataset'):
    """Generate empty dataset annotation template from schema."""
    all_schemas = get_all_schemas()
    schema_attributes = get_full_schema(dataset_type, all_schemas)

    template = {}
    for attr_name, attr_def in schema_attributes.items():
        if attr_def.get('multivalued', False):
            template[attr_name] = ['']
        else:
            template[attr_name] = ''

    template['_dataset_type'] = dataset_type
    template['_schema_source'] = 'data-model'
    template['_created_timestamp'] = datetime.now().isoformat()

    return template


def merge_annotations_into_template(template, existing_annotations):
    """
    Merge existing Synapse annotations into a template.

    Args:
        template: Template dict with all schema fields
        existing_annotations: Existing annotations from Synapse

    Returns:
        Merged dict with existing values preserved and template fields for missing values
    """
    # Start with existing annotations to preserve all current values
    merged = existing_annotations.copy()

    # Add any template fields that don't exist in existing annotations
    for key, value in template.items():
        if key not in merged:
            merged[key] = value

    return merged


def generate_annotation_files_with_merging(
    matched, new_only, 
    release_annotations, staging_annotations,
    output_file, dataset_name
):
    """
    Generate annotation JSON files with three-way merging: release > staging > template.

    Args:
        matched: List of (staging_syn_id, release_syn_id, local_path, form_name)
        new_only: List of (staging_syn_id, filename, form_name)
        release_annotations: Dict of {syn_id: {filename: {annotations}}} from release
        staging_annotations: Dict of {syn_id: {filename: {annotations}}} from staging
        output_file: Path to output JSON file
        dataset_name: Name of dataset (for display)
    """
    annotations_dict = {}
    template = create_file_annotation_template('ClinicalFile')

    # Process matched files (merge annotations)
    for staging_syn_id, release_syn_id, local_path, form_name in matched:
        # Get release annotations (highest priority)
        old_annot = {}
        if release_syn_id in release_annotations:
            release_file_data = release_annotations[release_syn_id]
            old_annot = list(release_file_data.values())[0]
        
        # Get staging annotations (if using Synapse workflow)
        new_annot = {}
        if staging_syn_id and staging_syn_id in staging_annotations:
            staging_file_data = staging_annotations[staging_syn_id]
            new_annot = list(staging_file_data.values())[0]
        
        # Merge: release > staging > template
        merged_annot = merge_file_annotations(old_annot, new_annot, template)
        
        # Use release syn_id as key (we're updating existing entity)
        annotations_dict[release_syn_id] = {form_name: merged_annot}

    # Process new files (template only, but keep any staging annotations)
    for staging_syn_id, filename, form_name in new_only:
        if staging_syn_id:
            # File exists in staging - use its annotations + template
            if staging_syn_id in staging_annotations:
                staging_file_data = staging_annotations[staging_syn_id]
                staging_annot = list(staging_file_data.values())[0]
                merged_annot = merge_annotations_into_template(template, staging_annot)
            else:
                merged_annot = template.copy()
            
            annotations_dict[staging_syn_id] = {form_name: merged_annot}
        else:
            # Local file only - use placeholder ID + template
            clean_name = extract_clean_filename(filename)
            placeholder_id = f"NEW_{clean_name}"
            annotations_dict[placeholder_id] = {form_name: template.copy()}

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(annotations_dict, f, indent=2)

    print(f"✓ Generated {output_file}")
    print(f"  - {len(matched)} matched files (with merged annotations)")
    print(f"  - {len(new_only)} new files")


# Generate annotation files
print("=" * 60)
print("GENERATING ANNOTATION TEMPLATES")
print("=" * 60)

print("\nGenerating ASSESS file annotations...")
generate_annotation_files_with_merging(
    assess_matched, assess_new_only,
    assess_existing_annotations, assess_staging_files,
    ASSESS_ANNOTATIONS_FILE, 'ASSESS'
)

print("\nGenerating PREVENT file annotations...")
generate_annotation_files_with_merging(
    prevent_matched, prevent_new_only,
    prevent_existing_annotations, prevent_staging_files,
    PREVENT_ANNOTATIONS_FILE, 'PREVENT'
)

print("\nGenerating dataset annotation templates...")

# Generate ASSESS dataset annotations (merge with existing)
assess_dataset_template = create_dataset_annotation_template('ClinicalDataset')
assess_dataset_merged = merge_annotations_into_template(assess_dataset_template, assess_dataset_existing_annotations)
with open(ASSESS_DATASET_ANNOTATIONS_FILE, 'w') as f:
    json.dump(assess_dataset_merged, f, indent=2)
print(f"✓ Generated {ASSESS_DATASET_ANNOTATIONS_FILE}")
print(f"  - Merged {len(assess_dataset_existing_annotations)} existing fields into template")

# Generate PREVENT dataset annotations (merge with existing)
prevent_dataset_template = create_dataset_annotation_template('ClinicalDataset')
prevent_dataset_merged = merge_annotations_into_template(prevent_dataset_template, prevent_dataset_existing_annotations)
with open(PREVENT_DATASET_ANNOTATIONS_FILE, 'w') as f:
    json.dump(prevent_dataset_merged, f, indent=2)
print(f"✓ Generated {PREVENT_DATASET_ANNOTATIONS_FILE}")
print(f"  - Merged {len(prevent_dataset_existing_annotations)} existing fields into template")

print("\n" + "=" * 60)
print("ANNOTATION FILES CREATED")
print("=" * 60)
print(f"Files created in: {ANNOTATIONS_DIR}")
print("  - assess_file_annotations.json")
print("  - prevent_file_annotations.json")
print("  - assess_dataset_annotations.json")
print("  - prevent_dataset_annotations.json")
print("\nAnnotation merging strategy:")
if USE_SYNAPSE_STAGING:
    print("  Priority: Release > Staging > Template")
else:
    print("  Priority: Release > Template (local file workflow)")
print("=" * 60)


GENERATING ANNOTATION TEMPLATES

Generating ASSESS file annotations...
✓ Loaded 54 schema definitions
✓ Generated /home/ramayyala/Documents/data-model/annotations/all_als/assess_file_annotations.json
  - 30 matched files (with merged annotations)
  - 2 new files

Generating PREVENT file annotations...
✓ Loaded 54 schema definitions
✓ Generated /home/ramayyala/Documents/data-model/annotations/all_als/prevent_file_annotations.json
  - 42 matched files (with merged annotations)
  - 8 new files

Generating dataset annotation templates...
✓ Loaded 54 schema definitions
✓ Generated /home/ramayyala/Documents/data-model/annotations/all_als/assess_dataset_annotations.json
  - Merged 12 existing fields into template
✓ Loaded 54 schema definitions
✓ Generated /home/ramayyala/Documents/data-model/annotations/all_als/prevent_dataset_annotations.json
  - Merged 11 existing fields into template

ANNOTATION FILES CREATED
Files created in: /home/ramayyala/Documents/data-model/annotations/all_als
  - as

## Step 8: Manual Annotation Editing

⏸️ **PAUSE HERE - Manual Editing Required**

Before proceeding to the next cell:

1. Navigate to: `/home/ramayyala/Documents/data-model/annotations/all_als/`
2. Edit all 4 annotation JSON files with proper metadata:
   - `assess_file_annotations.json`
   - `prevent_file_annotations.json`
   - `assess_dataset_annotations.json`
   - `prevent_dataset_annotations.json`

3. **Required fields** (must be filled):
   - `title` (string)
   - `creator` (list of strings)
   - `keywords` (list of strings)
   - `source` (string)
   - `url` (string)

4. **Optional but recommended**:
   - `description`, `dataType`, `species`, `disease`, etc.

5. For files with placeholder IDs (starting with "NEW_"):
   - Either create the entity in Synapse first and replace with real syn_id
   - Or skip these files in the upload step

**Once editing is complete, save all files and proceed to the next cell for validation.**


## Step 9: Load and Validate Annotations

Load the edited annotation files and validate them against the data model schemas.

**Validation must pass before proceeding to upload.**


In [19]:
def load_annotation_file(file_path):
    """Load annotation JSON file."""
    if not os.path.exists(file_path):
        print(f"⚠️  Warning: {file_path} not found")
        return {}

    with open(file_path, 'r') as f:
        return json.load(f)


def validate_all_annotations(file_annotations, dataset_annotations, dataset_name):
    """
    Validate all annotations for a dataset.
    Returns: (is_valid, all_errors, warnings, valid_count)
    """
    all_schemas = get_all_schemas()
    errors_by_file = {}
    warnings_by_file = {}
    valid_count = 0

    # Validate file annotations
    for syn_id, file_data in file_annotations.items():
        for filename, annotations in file_data.items():
            file_type = annotations.get('_file_type', 'ClinicalFile')

            is_valid, errors, warnings = validate_annotation_against_schema(
                annotations, file_type, all_schemas
            )

            if errors:
                errors_by_file[f"{syn_id}/{filename}"] = errors
            if warnings:
                warnings_by_file[f"{syn_id}/{filename}"] = warnings

            if is_valid:
                valid_count += 1

    # Validate dataset annotations
    dataset_type = dataset_annotations.get('_dataset_type', 'ClinicalDataset')
    is_valid, errors, warnings = validate_annotation_against_schema(
        dataset_annotations, dataset_type, all_schemas
    )

    if errors:
        errors_by_file[f"{dataset_name} Dataset"] = errors
    if warnings:
        warnings_by_file[f"{dataset_name} Dataset"] = warnings

    return (len(errors_by_file) == 0, errors_by_file, warnings_by_file, valid_count)


# Load annotations
print("=" * 60)
print("LOADING AND VALIDATING ANNOTATIONS")
print("=" * 60)

print("\nLoading annotation files...")
assess_file_annotations = load_annotation_file(ASSESS_ANNOTATIONS_FILE)
assess_dataset_annotations = load_annotation_file(ASSESS_DATASET_ANNOTATIONS_FILE)
prevent_file_annotations = load_annotation_file(PREVENT_ANNOTATIONS_FILE)
prevent_dataset_annotations = load_annotation_file(PREVENT_DATASET_ANNOTATIONS_FILE)

print(f"✓ Loaded ASSESS file annotations: {len(assess_file_annotations)} files")
print(f"✓ Loaded PREVENT file annotations: {len(prevent_file_annotations)} files")

# Validate annotations
print("\nValidating ASSESS annotations...")
assess_valid, assess_errors, assess_warnings, assess_valid_count = validate_all_annotations(
    assess_file_annotations, assess_dataset_annotations, 'ASSESS'
)

print(f"\nValidating PREVENT annotations...")
prevent_valid, prevent_errors, prevent_warnings, prevent_valid_count = validate_all_annotations(
    prevent_file_annotations, prevent_dataset_annotations, 'PREVENT'
)

# Report results
print("\n" + "=" * 60)
print("VALIDATION RESULTS")
print("=" * 60)

print(f"\nASSESS:")
if assess_valid:
    print(f"  ✓ All annotations valid ({assess_valid_count} files)")
else:
    print(f"  ✗ Validation failed ({len(assess_errors)} files with errors)")
    print("\n  Errors:")
    for file_id, errors in assess_errors.items():
        print(f"    - {file_id}:")
        for error in errors:
            print(f"      • {error}")

if assess_warnings:
    print(f"\n  ⚠️  Warnings ({len(assess_warnings)} files):")
    for file_id, warnings in assess_warnings.items():
        print(f"    - {file_id}:")
        for warning in warnings:
            print(f"      • {warning}")

print(f"\nPREVENT:")
if prevent_valid:
    print(f"  ✓ All annotations valid ({prevent_valid_count} files)")
else:
    print(f"  ✗ Validation failed ({len(prevent_errors)} files with errors)")
    print("\n  Errors:")
    for file_id, errors in prevent_errors.items():
        print(f"    - {file_id}:")
        for error in errors:
            print(f"      • {error}")

if prevent_warnings:
    print(f"\n  ⚠️  Warnings ({len(prevent_warnings)} files):")
    for file_id, warnings in prevent_warnings.items():
        print(f"    - {file_id}:")
        for warning in warnings:
            print(f"      • {warning}")

print("\n" + "=" * 60)

if not (assess_valid and prevent_valid):
    print("❌ VALIDATION FAILED - Please fix errors before proceeding")
    print("   Edit the annotation files and re-run this cell")
else:
    print("✅ VALIDATION PASSED - Ready to proceed to file renaming")

print("=" * 60)


LOADING AND VALIDATING ANNOTATIONS

Loading annotation files...
✓ Loaded ASSESS file annotations: 30 files
✓ Loaded PREVENT file annotations: 48 files

Validating ASSESS annotations...
✓ Loaded 54 schema definitions

Validating PREVENT annotations...
✓ Loaded 54 schema definitions

VALIDATION RESULTS

ASSESS:
  ✓ All annotations valid (30 files)

  ⚠️  Warnings (3 files):
    - syn68905760/ALSFRS-R:
      • Field administrationMode is a list but should be a single value
    - syn68905758/ALSFRS-R and Speech Anchor Questions:
      • Field administrationMode is a list but should be a single value
    - ASSESS Dataset:
      • Field title is a list but should be a single value
      • Field source is a list but should be a single value
      • Field disease is a list but should be a single value
      • Field studyType is a list but should be a single value
      • Field individualCount is a list but should be a single value

PREVENT:
  ✓ All annotations valid (48 files)

  ⚠️  Warnings 

## Step 10: Rename Files Using Form Name Mappings

Create staging directory with renamed files for upload:
- `v_ALLALS_AS_ASSEDEMOG.csv` → `Demographics.csv`

Files are copied (not moved) to staging directory.


In [20]:
def download_matched_files(syn, matched, staging_dir, download_dir, dataset_name):
    """
    Download new file versions from Synapse staging folder.
    
    Args:
        syn: Synapse client
        matched: List of (staging_syn_id, release_syn_id, local_path, form_name)
        staging_dir: Directory for downloaded files
        download_dir: Directory to download to
        dataset_name: Name of dataset (for logging)
    
    Returns: Dict[form_name → downloaded_path or local_path]
    """
    file_paths = {}
    success_count = 0
    error_count = 0
    local_count = 0
    
    for staging_syn_id, release_syn_id, local_path, form_name in matched:
        if staging_syn_id:
            # Download from Synapse staging folder
            downloaded_path = download_synapse_file(syn, staging_syn_id, download_dir)
            
            if downloaded_path:
                file_paths[form_name] = downloaded_path
                success_count += 1
            elif local_path:
                # Fallback to local file
                print(f"  ⚠️  Using local fallback for {form_name}")
                file_paths[form_name] = local_path
                local_count += 1
            else:
                print(f"  ✗ No file available for {form_name}")
                error_count += 1
        else:
            # Use local file (local workflow)
            if local_path:
                file_paths[form_name] = local_path
                local_count += 1
            else:
                print(f"  ✗ No local file for {form_name}")
                error_count += 1
    
    print(f"\n  Downloaded: {success_count}, Local: {local_count}, Errors: {error_count}")
    return file_paths


def create_renamed_staging_directory(file_paths, staging_dir, dataset_name):
    """
    Copy and rename files to staging directory.
    
    Args:
        file_paths: Dict[form_name → file_path]
        staging_dir: Directory to copy renamed files to
        dataset_name: Name of dataset (for logging)
    
    Returns: Dict[form_name → (renamed_filename, staged_path)]
    """
    rename_map = {}
    success_count = 0
    error_count = 0

    # Clear staging directory
    if os.path.exists(staging_dir):
        shutil.rmtree(staging_dir)
    os.makedirs(staging_dir, exist_ok=True)

    for form_name, source_path in file_paths.items():
        try:
            # Determine renamed filename
            # Sanitize form name to comply with Synapse naming rules
            sanitized_form_name = sanitize_synapse_filename(form_name)
            renamed_filename = f"{sanitized_form_name}.csv"
            staged_path = os.path.join(staging_dir, renamed_filename)

            # Copy file to staging
            shutil.copy2(source_path, staged_path)

            rename_map[form_name] = (renamed_filename, staged_path)
            success_count += 1

            if VERBOSE:
                print(f"  ✓ {os.path.basename(source_path)} → {renamed_filename}")

        except Exception as e:
            print(f"  ✗ Error renaming {form_name}: {e}")
            error_count += 1

    print(f"\n  Total: {success_count} files renamed, {error_count} errors")
    return rename_map


# Download and rename files
print("=" * 60)
print("DOWNLOADING AND RENAMING FILES")
print("=" * 60)

# Connect to Synapse (needed for downloads even in DRY_RUN)
if USE_SYNAPSE_STAGING:
    if 'syn' not in globals() or syn is None:
        syn = connect_to_synapse()
else:
    syn = None

# Download ASSESS files (if using Synapse staging)
if USE_SYNAPSE_STAGING:
    print("\nDownloading ASSESS files from staging folder...")
    assess_file_paths = download_matched_files(
        syn, assess_matched, ASSESS_STAGING_DIR, ASSESS_DOWNLOAD_DIR, 'ASSESS'
    )
else:
    print("\nUsing local ASSESS files...")
    assess_file_paths = {}
    for _, release_syn_id, local_path, form_name in assess_matched:
        if local_path:
            assess_file_paths[form_name] = local_path

print(f"\nRenaming ASSESS files...")
assess_rename_map = create_renamed_staging_directory(
    assess_file_paths, ASSESS_STAGING_DIR, 'ASSESS'
)

# Download PREVENT files (if using Synapse staging)
if USE_SYNAPSE_STAGING:
    print("\nDownloading PREVENT files from staging folder...")
    prevent_file_paths = download_matched_files(
        syn, prevent_matched, PREVENT_STAGING_DIR, PREVENT_DOWNLOAD_DIR, 'PREVENT'
    )
else:
    print("\nUsing local PREVENT files...")
    prevent_file_paths = {}
    for _, release_syn_id, local_path, form_name in prevent_matched:
        if local_path:
            prevent_file_paths[form_name] = local_path

print(f"\nRenaming PREVENT files...")
prevent_rename_map = create_renamed_staging_directory(
    prevent_file_paths, PREVENT_STAGING_DIR, 'PREVENT'
)

print("\n" + "=" * 60)
print("FILE PREPARATION COMPLETE")
print("=" * 60)
print(f"ASSESS staged files: {len(assess_rename_map)}")
print(f"  Location: {ASSESS_STAGING_DIR}")
print(f"\nPREVENT staged files: {len(prevent_rename_map)}")
print(f"  Location: {PREVENT_STAGING_DIR}")
if USE_SYNAPSE_STAGING:
    print(f"\nDownloaded files location:")
    print(f"  - ASSESS: {ASSESS_DOWNLOAD_DIR}")
    print(f"  - PREVENT: {PREVENT_DOWNLOAD_DIR}")
print("=" * 60)


DOWNLOADING AND RENAMING FILES



[syn71824511]: Downloaded to /home/ramayyala/Documents/data-model/downloads/all_als/assess/v_ALLALS_AS_ASSEAELOG.csv


  ✓ Downloaded v_ALLALS_AS_ASSEAELOG.csv to /home/ramayyala/Documents/data-model/downloads/all_als/assess/v_ALLALS_AS_ASSEAELOG.csv


[syn71824519]: Downloaded to /home/ramayyala/Documents/data-model/downloads/all_als/assess/v_ALLALS_AS_ASSEALSFRSR.csv


  ✓ Downloaded v_ALLALS_AS_ASSEALSFRSR.csv to /home/ramayyala/Documents/data-model/downloads/all_als/assess/v_ALLALS_AS_ASSEALSFRSR.csv


...

  ✓ Downloaded v_ALLALS_PR_PREVVC.csv to /home/ramayyala/Documents/data-model/downloads/all_als/prevent/v_ALLALS_PR_PREVVC.csv


[syn71824504]: Downloaded to /home/ramayyala/Documents/data-model/downloads/all_als/prevent/v_ALLALS_PR_PREVVISTYPE.csv


  ✓ Downloaded v_ALLALS_PR_PREVVISTYPE.csv to /home/ramayyala/Documents/data-model/downloads/all_als/prevent/v_ALLALS_PR_PREVVISTYPE.csv


[syn71824505]: Downloaded to /home/ramayyala/Documents/data-model/downloads/all_als/prevent/v_ALLALS_PR_PREVWEIGHT.csv


  ✓ Downloaded v_ALLALS_PR_PREVWEIGHT.csv to /home/ramayyala/Documents/data-model/downloads/all_als/prevent/v_ALLALS_PR_PREVWEIGHT.csv

  Downloaded: 42, Local: 0, Errors: 0

Renaming PREVENT files...
  ✓ v_ALLALS_PR_PREVAELOG.csv → Adverse Event (AE) Log.csv
  ✓ v_ALLALS_PR_PREVBLDCOLINC.csv → Blood Collection (PREVENT InClinic).csv
  ✓ v_ALLALS_PR_PREVBLDCOLREM.csv → Blood Collection (PREVENT Remote).csv
  ✓ v_ALLALS_PR_PREVCONMD.csv → Concomitant Medication Log.csv
  ✓ v_ALLALS_PR_PREVCSFCO.csv → Cerebral Spinal Fluid (CSF) Collection.csv
  ✓ v_ALLALS_PR_PREVDEMOG.csv → Demographics.csv
  ✓ v_ALLALS_PR_PREVDSA.csv → Digital Speech Assessment.csv
  ✓ v_ALLALS_PR_PREVDSAUI.csv → Digital Speech Assessment User Information.csv
  ✓ v_ALLALS_PR_PREVECAS.csv → ECAS (Edinburgh Cognitive and Behavioral ALS Screen).csv
  ✓ v_ALLALS_PR_PREVECASCGI.csv → ECAS (Edinburgh Cognitive and Behavioral ALS Screen) Caregiver Interview.csv
  ✓ v_ALLALS_PR_PREVELCONGEN.csv → Enrollment Confirmation (PREVE

In [21]:

def upload_file_version(syn, syn_id, file_path, filename, version_label, version_comment, annotations, dry_run=True):
    """
    Upload a new version of a file to Synapse.
    Returns: (success, error_message)
    """
    try:
        # Clean annotations (remove metadata)
        cleaned_annotations = clean_annotations_for_synapse(annotations)

        if dry_run:
            print(f"  [DRY_RUN] Would upload:")
            print(f"    - File: {file_path}")
            print(f"    - To: {syn_id}")
            print(f"    - Version: {version_label}")
            print(f"    - Annotations: {len(cleaned_annotations)} fields")
            return (True, None)

        # Create File object with annotations
        file_entity = File(
            path=file_path,
            id=syn_id,
            version_comment=version_comment,
            version_label=version_label,
            annotations=cleaned_annotations
        )

        # Upload to Synapse using new API
        file_entity = file_entity.store()

        if VERBOSE:
            print(f"  ✓ Uploaded {filename} as version {version_label}")

        return (True, None)

    except Exception as e:
        error_msg = f"Failed to upload {filename}: {e}"
        print(f"  ✗ {error_msg}")
        return (False, error_msg)


def process_file_uploads(syn, file_annotations, rename_map, matched, dataset_name, dry_run=True):
    """
    Process all file uploads for matched files (creates new versions).
    Returns: (success_count, error_count, skipped_count, results)
    """
    success_count = 0
    error_count = 0
    skipped_count = 0
    results = []

    for staging_syn_id, release_syn_id, local_path, form_name in matched:
        # Get staged file path
        if form_name not in rename_map:
            print(f"  ⚠️  Skipping {form_name} (not found in rename map)")
            skipped_count += 1
            continue

        renamed_filename, staged_path = rename_map[form_name]

        # Get annotations for this file
        if release_syn_id not in file_annotations:
            print(f"  ⚠️  Skipping {form_name} (no annotations found)")
            skipped_count += 1
            continue

        file_data = file_annotations[release_syn_id]
        annotations = list(file_data.values())[0]  # Get first (and only) annotation dict

        # Upload file to RELEASE syn_id (creates new version)
        success, error = upload_file_version(
            syn, release_syn_id, staged_path, renamed_filename,
            VERSION_LABEL, VERSION_COMMENT, annotations, dry_run
        )

        if success:
            success_count += 1
        else:
            error_count += 1

        results.append({
            'filename': renamed_filename,
            'syn_id': release_syn_id,
            'success': success,
            'error': error
        })

    return success_count, error_count, skipped_count, results


def process_new_file_moves(
    syn, file_annotations, new_only, 
    release_folder_syn_id, dataset_syn_id, 
    dataset_name, dry_run=True
):
    """
    Process moving new files from staging folder to release folder.
    Returns: (success_count, error_count, skipped_count, results)
    """
    success_count = 0
    error_count = 0
    skipped_count = 0
    results = []

    for staging_syn_id, filename, form_name in new_only:
        if not staging_syn_id:
            # Local file only - needs manual entity creation
            print(f"  ⚠️  Skipping {filename} (local file - create entity first)")
            skipped_count += 1
            continue

        # Get annotations for this file
        if staging_syn_id not in file_annotations:
            print(f"  ⚠️  Skipping {filename} (no annotations found)")
            skipped_count += 1
            continue

        file_data = file_annotations[staging_syn_id]
        annotations = list(file_data.values())[0]

        # Move file to release folder + add to dataset + set annotations
        success, error = move_file_to_release(
            syn, staging_syn_id, release_folder_syn_id, 
            dataset_syn_id, annotations, form_name, dry_run
        )

        if success:
            success_count += 1
        else:
            error_count += 1

        results.append({
            'filename': filename,
            'syn_id': staging_syn_id,
            'success': success,
            'error': error
        })

    return success_count, error_count, skipped_count, results


# Upload and move files
print("=" * 60)
print(f"UPLOADING FILES AND MOVING NEW FILES (DRY_RUN={DRY_RUN})")
print("=" * 60)

if DRY_RUN:
    print("\n⚠️  DRY_RUN MODE - No files will be uploaded or moved")
    print("Set DRY_RUN=False in configuration to actually execute\n")

# Connect to Synapse (needed even in DRY_RUN for file info)
if 'syn' not in globals() or syn is None:
    syn = connect_to_synapse()

# ========== ASSESS ==========
print("\n--- ASSESS Dataset ---")

# Upload matched files (update with new versions)
print(f"\nUploading ASSESS matched files...")
assess_upload_success, assess_upload_errors, assess_upload_skipped, assess_upload_results = process_file_uploads(
    syn, assess_file_annotations, assess_rename_map,
    assess_matched, 'ASSESS', DRY_RUN
)

# Move new files (if using Synapse staging)
assess_move_success = 0
assess_move_errors = 0
assess_move_skipped = 0
assess_move_results = []

if USE_SYNAPSE_STAGING and assess_new_only:
    print(f"\nMoving ASSESS new files...")
    assess_move_success, assess_move_errors, assess_move_skipped, assess_move_results = process_new_file_moves(
        syn, assess_file_annotations, assess_new_only,
        ASSESS_RELEASE_FOLDER_SYN_ID, ASSESS_DATASET_SYN_ID,
        'ASSESS', DRY_RUN
    )
elif assess_new_only:
    print(f"\n⚠️  {len(assess_new_only)} new ASSESS files (local workflow - create entities manually)")

# ========== PREVENT ==========
print("\n--- PREVENT Dataset ---")

# Upload matched files (update with new versions)
print(f"\nUploading PREVENT matched files...")
prevent_upload_success, prevent_upload_errors, prevent_upload_skipped, prevent_upload_results = process_file_uploads(
    syn, prevent_file_annotations, prevent_rename_map,
    prevent_matched, 'PREVENT', DRY_RUN
)

# Move new files (if using Synapse staging)
prevent_move_success = 0
prevent_move_errors = 0
prevent_move_skipped = 0
prevent_move_results = []

if USE_SYNAPSE_STAGING and prevent_new_only:
    print(f"\nMoving PREVENT new files...")
    prevent_move_success, prevent_move_errors, prevent_move_skipped, prevent_move_results = process_new_file_moves(
        syn, prevent_file_annotations, prevent_new_only,
        PREVENT_RELEASE_FOLDER_SYN_ID, PREVENT_DATASET_SYN_ID,
        'PREVENT', DRY_RUN
    )
elif prevent_new_only:
    print(f"\n⚠️  {len(prevent_new_only)} new PREVENT files (local workflow - create entities manually)")

# ========== SUMMARY ==========
print("\n" + "=" * 60)
print("OPERATION RESULTS")
print("=" * 60)

print(f"\nASSESS:")
print(f"  Uploads (new versions):")
print(f"    ✓ Success: {assess_upload_success}")
print(f"    ✗ Errors: {assess_upload_errors}")
print(f"    ⚠️  Skipped: {assess_upload_skipped}")
if USE_SYNAPSE_STAGING:
    print(f"  Moves (new files):")
    print(f"    ✓ Success: {assess_move_success}")
    print(f"    ✗ Errors: {assess_move_errors}")
    print(f"    ⚠️  Skipped: {assess_move_skipped}")

print(f"\nPREVENT:")
print(f"  Uploads (new versions):")
print(f"    ✓ Success: {prevent_upload_success}")
print(f"    ✗ Errors: {prevent_upload_errors}")
print(f"    ⚠️  Skipped: {prevent_upload_skipped}")
if USE_SYNAPSE_STAGING:
    print(f"  Moves (new files):")
    print(f"    ✓ Success: {prevent_move_success}")
    print(f"    ✗ Errors: {prevent_move_errors}")
    print(f"    ⚠️  Skipped: {prevent_move_skipped}")

# Show any errors
all_results = assess_upload_results + assess_move_results + prevent_upload_results + prevent_move_results
error_results = [r for r in all_results if not r['success']]

if error_results:
    print("\n❌ Errors:")
    for result in error_results:
        print(f"  - {result['filename']}: {result['error']}")

print("\n" + "=" * 60)
if DRY_RUN:
    print("✓ DRY_RUN COMPLETE - Set DRY_RUN=False to actually execute")
elif not error_results:
    print("✅ ALL OPERATIONS SUCCESSFUL")
else:
    print("⚠️  OPERATIONS COMPLETED WITH ERRORS")

print("=" * 60)


UPLOADING FILES AND MOVING NEW FILES (DRY_RUN=False)

--- ASSESS Dataset ---

Uploading ASSESS matched files...


Uploading to Synapse storage: 100%|██████████| 46.2k/46.2k [00:00<00:00, 132kB/s, Adverse Event (AE) Log.csv]


  ✓ Uploaded Adverse Event (AE) Log.csv as version v3-DEC


Uploading to Synapse storage: 100%|██████████| 164k/164k [00:00<00:00, 299kB/s, ALSFRSR.csv]


  ✓ Uploaded ALSFRSR.csv as version v3-DEC


...

  ✓ Uploaded Vital Capacity (VC).csv as version v3-DEC


Uploading to Synapse storage: 100%|██████████| 67.6k/67.6k [00:00<00:00, 185kB/s, Visit Type.csv]


  ✓ Uploaded Visit Type.csv as version v3-DEC


Uploading to Synapse storage: 100%|██████████| 12.6k/12.6k [00:00<00:00, 34.7kB/s, Weight.csv]


  ✓ Uploaded Weight.csv as version v3-DEC

Moving PREVENT new files...
  ✓ Moved v_ALLALS_PR_NPROREG.csv to release folder
  ✓ Renamed v_ALLALS_PR_NPROREG.csv → NPROREG.csv
  ✓ Added v_ALLALS_PR_NPROREG.csv to dataset
  ✓ Set annotations on v_ALLALS_PR_NPROREG.csv
  ✓ Moved v_ALLALS_PR_PREVFUTELEV.csv to release folder
  ✓ Renamed v_ALLALS_PR_PREVFUTELEV.csv → FollowUp Telephone Visit.csv
  ✓ Added v_ALLALS_PR_PREVFUTELEV.csv to dataset
  ✓ Set annotations on v_ALLALS_PR_PREVFUTELEV.csv
  ✓ Moved v_ALLALS_PR_PREVHHD.csv to release folder
  ✓ Renamed v_ALLALS_PR_PREVHHD.csv → Handheld Dynamometry (HHD).csv
  ✓ Added v_ALLALS_PR_PREVHHD.csv to dataset
  ✓ Set annotations on v_ALLALS_PR_PREVHHD.csv
  ✓ Moved v_ALLALS_PR_PREVICFGEN.csv to release folder
  ✓ Renamed v_ALLALS_PR_PREVICFGEN.csv → Informed Consent Log Genetic Testing SubStudy.csv
  ✓ Added v_ALLALS_PR_PREVICFGEN.csv to dataset
  ✓ Set annotations on v_ALLALS_PR_PREVICFGEN.csv
  ✓ Moved v_ALLALS_PR_PREVOGCVH.csv to release fold

## Step 12: Verify Uploads

Verify successful uploads by checking file versions and annotations in Synapse.


In [22]:
def verify_file_version(syn, syn_id, expected_version_label):
    """
    Verify that a file has the expected version label.
    Returns: (success, actual_version)
    """
    try:
        entity = syn.get(syn_id, downloadFile=False)
        actual_version = entity.versionLabel if hasattr(entity, 'versionLabel') else None

        if actual_version == expected_version_label:
            return (True, actual_version)
        else:
            return (False, actual_version)

    except Exception as e:
        if VERBOSE:
            print(f"  ⚠️  Could not verify {syn_id}: {e}")
        return (False, None)


def verify_file_parent(syn, syn_id, expected_parent_id):
    """
    Verify that a file has the expected parent folder.
    Returns: (success, actual_parent_id)
    """
    try:
        entity = syn.get(syn_id, downloadFile=False)
        actual_parent = entity.parentId if hasattr(entity, 'parentId') else None

        if actual_parent == expected_parent_id:
            return (True, actual_parent)
        else:
            return (False, actual_parent)

    except Exception as e:
        if VERBOSE:
            print(f"  ⚠️  Could not verify parent for {syn_id}: {e}")
        return (False, None)


def verify_file_in_dataset(syn, dataset_syn_id, file_syn_id):
    """
    Verify that a file is in the dataset.
    Returns: (success, is_in_dataset)
    """
    try:
        dataset = Dataset(dataset_syn_id).get()
        file_ids = [item.id for item in dataset.items]
        
        is_in_dataset = file_syn_id in file_ids
        return (is_in_dataset, is_in_dataset)

    except Exception as e:
        if VERBOSE:
            print(f"  ⚠️  Could not verify dataset membership: {e}")
        return (False, False)


def verify_uploads(syn, matched, expected_version_label, dataset_name):
    """
    Verify all uploads for a dataset.
    Returns: (verified_count, errors)
    """
    verified_count = 0
    errors = []

    for staging_syn_id, release_syn_id, local_path, form_name in matched:
        success, actual_version = verify_file_version(syn, release_syn_id, expected_version_label)

        if success:
            verified_count += 1
            if VERBOSE:
                print(f"  ✓ {form_name}: version {actual_version}")
        else:
            error_msg = f"{form_name}: expected {expected_version_label}, got {actual_version}"
            errors.append(error_msg)
            print(f"  ✗ {error_msg}")

    return verified_count, errors


def verify_moves(syn, new_only, release_folder_syn_id, dataset_syn_id, dataset_name):
    """
    Verify all file moves for a dataset.
    Returns: (verified_count, parent_errors, dataset_errors)
    """
    verified_count = 0
    parent_errors = []
    dataset_errors = []

    for staging_syn_id, filename, form_name in new_only:
        if not staging_syn_id:
            continue  # Skip local-only files
        
        # Verify parent folder
        parent_success, actual_parent = verify_file_parent(syn, staging_syn_id, release_folder_syn_id)
        
        if not parent_success:
            error_msg = f"{form_name}: expected parent {release_folder_syn_id}, got {actual_parent}"
            parent_errors.append(error_msg)
            print(f"  ✗ Parent: {error_msg}")
        
        # Verify dataset membership
        dataset_success, in_dataset = verify_file_in_dataset(syn, dataset_syn_id, staging_syn_id)
        
        if not dataset_success or not in_dataset:
            error_msg = f"{form_name}: not in dataset {dataset_syn_id}"
            dataset_errors.append(error_msg)
            print(f"  ✗ Dataset: {error_msg}")
        
        # If both checks pass, count as verified
        if parent_success and dataset_success:
            verified_count += 1
            if VERBOSE:
                print(f"  ✓ {form_name}: moved and in dataset")

    return verified_count, parent_errors, dataset_errors


# Verify uploads and moves
print("=" * 60)
print("VERIFYING OPERATIONS")
print("=" * 60)

if DRY_RUN:
    print("\n⚠️  Skipping verification (DRY_RUN mode)")
    print("=" * 60)
else:
    print(f"\nConnecting to Synapse...")
    if 'syn' not in globals():
        syn = connect_to_synapse()

    # ========== ASSESS ==========
    print("\n--- ASSESS Dataset ---")
    
    # Verify uploads (new versions)
    print(f"\nVerifying ASSESS file versions...")
    assess_verified, assess_version_errors = verify_uploads(
        syn, assess_matched, VERSION_LABEL, 'ASSESS'
    )

    # Verify moves (new files)
    assess_moves_verified = 0
    assess_parent_errors = []
    assess_dataset_errors = []
    
    if USE_SYNAPSE_STAGING and assess_new_only:
        print(f"\nVerifying ASSESS file moves...")
        assess_moves_verified, assess_parent_errors, assess_dataset_errors = verify_moves(
            syn, assess_new_only, ASSESS_RELEASE_FOLDER_SYN_ID, ASSESS_DATASET_SYN_ID, 'ASSESS'
        )

    # ========== PREVENT ==========
    print("\n--- PREVENT Dataset ---")
    
    # Verify uploads (new versions)
    print(f"\nVerifying PREVENT file versions...")
    prevent_verified, prevent_version_errors = verify_uploads(
        syn, prevent_matched, VERSION_LABEL, 'PREVENT'
    )

    # Verify moves (new files)
    prevent_moves_verified = 0
    prevent_parent_errors = []
    prevent_dataset_errors = []
    
    if USE_SYNAPSE_STAGING and prevent_new_only:
        print(f"\nVerifying PREVENT file moves...")
        prevent_moves_verified, prevent_parent_errors, prevent_dataset_errors = verify_moves(
            syn, prevent_new_only, PREVENT_RELEASE_FOLDER_SYN_ID, PREVENT_DATASET_SYN_ID, 'PREVENT'
        )

    # ========== SUMMARY ==========
    print("\n" + "=" * 60)
    print("VERIFICATION RESULTS")
    print("=" * 60)

    print(f"\nASSESS:")
    print(f"  Uploads verified: {assess_verified} files")
    if assess_version_errors:
        print(f"  ✗ Version errors: {len(assess_version_errors)}")
        for error in assess_version_errors:
            print(f"    - {error}")
    
    if USE_SYNAPSE_STAGING:
        print(f"  Moves verified: {assess_moves_verified} files")
        if assess_parent_errors:
            print(f"  ✗ Parent folder errors: {len(assess_parent_errors)}")
            for error in assess_parent_errors:
                print(f"    - {error}")
        if assess_dataset_errors:
            print(f"  ✗ Dataset membership errors: {len(assess_dataset_errors)}")
            for error in assess_dataset_errors:
                print(f"    - {error}")

    print(f"\nPREVENT:")
    print(f"  Uploads verified: {prevent_verified} files")
    if prevent_version_errors:
        print(f"  ✗ Version errors: {len(prevent_version_errors)}")
        for error in prevent_version_errors:
            print(f"    - {error}")
    
    if USE_SYNAPSE_STAGING:
        print(f"  Moves verified: {prevent_moves_verified} files")
        if prevent_parent_errors:
            print(f"  ✗ Parent folder errors: {len(prevent_parent_errors)}")
            for error in prevent_parent_errors:
                print(f"    - {error}")
        if prevent_dataset_errors:
            print(f"  ✗ Dataset membership errors: {len(prevent_dataset_errors)}")
            for error in prevent_dataset_errors:
                print(f"    - {error}")

    print("\n" + "=" * 60)

    # Overall status
    all_errors = (assess_version_errors + assess_parent_errors + assess_dataset_errors +
                  prevent_version_errors + prevent_parent_errors + prevent_dataset_errors)

    if not all_errors:
        print("✅ ALL OPERATIONS VERIFIED SUCCESSFULLY")
    else:
        print("⚠️  VERIFICATION COMPLETED WITH DISCREPANCIES")

    print("=" * 60)

# ========== FINAL WORKFLOW SUMMARY ==========
print("\n" + "=" * 60)
print("WORKFLOW COMPLETE")
print("=" * 60)
print(f"Summary:")
print(f"  - ASSESS files processed: {len(assess_matched)} updated")
if USE_SYNAPSE_STAGING:
    print(f"  - ASSESS new files: {len(assess_new_only)} moved")
print(f"  - PREVENT files processed: {len(prevent_matched)} updated")
if USE_SYNAPSE_STAGING:
    print(f"  - PREVENT new files: {len(prevent_new_only)} moved")
print(f"  - Version: {VERSION_LABEL}")
print(f"  - Workflow mode: {'Synapse-based' if USE_SYNAPSE_STAGING else 'Local files'}")
print(f"  - DRY_RUN: {DRY_RUN}")
print("=" * 60)


VERIFYING OPERATIONS

Connecting to Synapse...

--- ASSESS Dataset ---

Verifying ASSESS file versions...
  ✓ Adverse Event (AE) Log: version v3-DEC
  ✓ ALSFRS-R: version v3-DEC
  ✓ ALSFRS-R and Speech Anchor Questions: version v3-DEC
  ✓ ALS History: version v3-DEC
  ✓ Blood Collection (ASSESS): version v3-DEC
  ✓ Concomitant Medication Log: version v3-DEC
  ✓ Cerebral Spinal Fluid (CSF) Collection: version v3-DEC
  ✓ Demographics: version v3-DEC
  ✓ Digital Speech Assessment: version v3-DEC
  ✓ Digital Speech Assessment User Information: version v3-DEC
  ✓ ECAS (Edinburgh Cognitive and Behavioral ALS Screen): version v3-DEC
  ✓ ECAS (Edinburgh Cognitive and Behavioral ALS Screen) - Caregiver Interview: version v3-DEC
  ✓ Lumbar Puncture Eligibility Confirmation: version v3-DEC
  ✓ Enrollment Confirmation: version v3-DEC
  ✓ Eligibility Criteria (ASSESS Control): version v3-DEC
  ✓ Eligibility Criteria (ASSESS Symptomatic): version v3-DEC
  ✓ Lumbar Puncture Eligibility Criteria: vers